In [1]:
import pandas as pd
import numpy as np

VOCAB_SIZE = 2500 

TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

In [2]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype= int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype= int)

In [3]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5797, 2249,    0,    1],
       [5797, 2347,    0,    1],
       [5797, 2374,    0,    1]])

In [4]:
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5792, 1668,    0,    1],
       [5792, 2078,    0,    1],
       [5792, 2178,    0,    1]])

In [5]:
sparse_train_data.shape                                            # 4 => DOC_ID......WORD_ID......LABEL.......OCCURRENCE
                                                              # 259350 => total rows  or  total test words 

(259350, 4)

In [6]:
sparse_test_data.shape

(116879, 4)

In [7]:
np.unique(sparse_train_data[:,0]).size                    # Find the unique elements of an array.
                                                 # training size = 4058 but we lost 44 empty mails .... so it is 4014

4014

In [8]:
np.unique(sparse_test_data[:,0]).size           # testing size = 1740 but we lost 15 empty mails .... so it is 1725

1725

In [9]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
                                                                                  # DOC_ID => index number
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))        # CATEGORY => ham = 0  ,,, spam = 1
                                                                        
        
    doc_id_names = np.unique(sparse_matrix[:, 0])    
    
    
    full_matrix = pd.DataFrame(index =doc_id_names, columns =column_names) 
    
    
    full_matrix.fillna(value=0, inplace=True)  
    
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i][doc_idx]
        
        word_id = sparse_matrix[i][word_idx]
        
        label = sparse_matrix[i][cat_idx]
        
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [10]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)              # VOCAB_SIZE => 2500
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,1,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5793,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,2,2,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0


## According to Bayes formula

# $$ P(Spam|token) =\frac{P(token|Spam) \times P(Spam)}{P(token)} $$

# $$ P(ham|token) =\frac{P(token|ham) \times P(ham)}{P(token)} $$

In [11]:
full_train_data.CATEGORY.size

4014

In [12]:
full_train_data.CATEGORY.sum()               # this is equal to the addition of all spam 

1247

In [13]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam                                                                         # probability of spam email = 31.06 %

0.3106626806178376

In [14]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5793,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5794,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
email_lengths = full_train_features.sum(axis = 1)
email_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5793     46
5794     30
5795    228
5796     57
5797    264
Length: 4014, dtype: int64

In [16]:
email_lengths.shape[0]

4014

In [17]:
total_wc = email_lengths.sum()
total_wc                               # total word contain

429329

In [18]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]
spam_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
1885     45
1887     26
1889    115
1890     26
1895    162
Length: 1247, dtype: int64

In [19]:
spam_wc = spam_lengths.sum()
spam_wc                                              # total word contain in spam

176284

In [20]:
ham_lengths = email_lengths[full_train_data.CATEGORY ==0]
ham_lengths

DOC_ID
1896     82
1898     41
1899     41
1900    112
1901     66
       ... 
5793     46
5794     30
5795    228
5796     57
5797    264
Length: 2767, dtype: int64

In [21]:
nonspam_wc = ham_lengths.sum()
nonspam_wc                                     # total word contain in ham

253045

In [22]:
nonspam_wc + spam_wc

429329

In [23]:
nonspam_wc/ham_lengths.shape[0]                                    # average number of words in the ham mail

91.45102999638598

In [24]:
spam_wc/spam_lengths.shape[0]                        # averagwe number of words in the spam mail

141.3664795509222

In [25]:
train_spam_token = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_token                                                                  # tokenised word of spam

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
train_spam_token.shape

(1247, 2500)

### 10/0 = error ....&....0/10 = 0...... if there is any empty mails then we will get an error ..... thats why we add 1 

In [27]:
summed_spam_tokens = train_spam_token.sum(axis = 0) + 1                                      # Laplace Smoothing
summed_spam_tokens

0       2180
1        935
2       1219
3       2021
4       1215
        ... 
2495      28
2496      26
2497       2
2498      15
2499      22
Length: 2500, dtype: int64

In [28]:
prob_spam_tokens = summed_spam_tokens/(spam_wc + VOCAB_SIZE)
prob_spam_tokens                                                     # ( probability of each each tokenised word of spam emails)

0       0.012193
1       0.005230
2       0.006818
3       0.011304
4       0.006796
          ...   
2495    0.000157
2496    0.000145
2497    0.000011
2498    0.000084
2499    0.000123
Length: 2500, dtype: float64

In [29]:
prob_spam_tokens.sum()            # consider this 0.99999 = 1     ( probability of each each tokenised word of spam emails)

0.9999999999999999

In [30]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 
summed_ham_tokens

0       5366
1       2597
2       2043
3        933
4       1646
        ... 
2495      12
2496       4
2497      33
2498      14
2499       6
Length: 2500, dtype: int64

In [31]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam                                                 # ( probability of each each tokenised word of ham emails)

0       0.020998
1       0.010163
2       0.007995
3       0.003651
4       0.006441
          ...   
2495    0.000047
2496    0.000016
2497    0.000129
2498    0.000055
2499    0.000023
Length: 2500, dtype: float64

In [32]:
prob_tokens_nonspam.sum()                              # ( probability of each each tokenised word of ham emails)

1.0

In [33]:
prob_token_all = full_train_features.sum(axis = 0)/total_wc
prob_token_all                                                          # ( probability of each tokenised word of emails)

0       0.017572
1       0.008222
2       0.007593
3       0.006876
4       0.006659
          ...   
2495    0.000089
2496    0.000065
2497    0.000077
2498    0.000063
2499    0.000061
Length: 2500, dtype: float64

In [34]:
prob_token_all.sum()                                          # ( probability of all tokenised word of emails)

1.0

In [35]:
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

In [36]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_spam_tokens)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_token_all)

# prepare the test data....

In [37]:
sparse_test_data[:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 4],
       [8, 4, 1, 2],
       [8, 5, 1, 1],
       [8, 6, 1, 2]])

In [38]:
sparse_test_data.shape

(116879, 4)

In [39]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5782,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5785,0,2,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY'] 

In [41]:
X_test.shape

(1725, 2500)

In [42]:
y_test = full_test_data.CATEGORY
y_test.shape

(1725,)

In [43]:
TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

In [44]:
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)